# Setting Parameters

In [35]:
# Classes: '33+1', '8+1', '1+1'
apply_classes = ['33+1', '8+1', '1+1']

# Samplers: None, 'RandomOverSampler', 'RandomUnderSampler', 'SMOTE', 'ClusterCentroids'
# Note: SMOTE is O(n^2) to O(n^3). Using a sample size of 1 CSV file takes ~10 min per class to oversample on a OC'd 9700k.
#       Limit the sample size to 1 CSV file unless you want to measure completion time on a geologic timescale.
apply_sampling = 'ClusterCentroids'    # Select ONE from above

# Evaluators: 'XGBoost', 'LogisticRegression', 'Perceptron', 'AdaBoost', 'RandomForest', 'DeepNeuralNetwork', 'KNearestNeighbor'
apply_evaluators = ['Perceptron', 'AdaBoost']


# Notebook parameter validation
for _class in apply_classes:
    if _class not in ['33+1', '8+1', '1+1']:
        assert False, f'{_class} is an invalid class structure.'

if apply_sampling not in [None, 'RandomOverSampler', 'RandomUnderSampler', 'SMOTE', 'ClusterCentroids']:
    assert False, f'{apply_sampling} is an invalid sampling method.'
    
for evaluator in apply_evaluators:
    if evaluator not in ['XGBoost', 'LogisticRegression', 'Perceptron', 'AdaBoost', 
                         'RandomForest', 'DeepNeuralNetwork', 'KNearestNeighbor']:
        assert False, f'{evaluator} is an invalid evaluator.'

# Dataset Handling
## Common Imports

In [36]:
import os
import pandas as pd
import random
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from ydata_profiling import ProfileReport
from tqdm import tqdm

## Loading the Dataset

In [37]:
DATASET_DIRECTORY = '../dataset/'  # If your dataset is within your python project directory, change this to the relative path to your dataset
csv_filepaths = [filename for filename in os.listdir(DATASET_DIRECTORY) if filename.endswith('.csv')]

print(csv_filepaths)

# If there are more than X CSV files, randomly select X files from the list
sample_size = 5

if len(csv_filepaths) > sample_size:
    csv_filepaths = random.sample(csv_filepaths, sample_size)
    print(csv_filepaths)

csv_filepaths.sort()

# list of csv files used
data_sets = csv_filepaths

full_data = pd.DataFrame()
for data_set in data_sets:
    print(f"data set {data_set} out of {len(data_sets)} \n")
    data_path = os.path.join(DATASET_DIRECTORY, data_set)
    df = pd.read_csv(data_path)
    full_data = pd.concat([full_data, df])

# prints an instance of each class
print("Before encoding:")
unique_labels = full_data['label'].unique()
for label in unique_labels:
    print(f"First instance of {label}:")
    print(full_data[full_data['label'] == label].iloc[0])

# Shuffle data
full_data = shuffle(full_data, random_state=42)

# prove if the data is loaded properly
print("Real data:")
print(full_data[:2])
print(full_data.shape)

# Assuming 'label' is the column name for the labels in the DataFrame `synth_data`
unique_labels = full_data['label'].nunique()

# Print the number of unique labels
print(f"There are {unique_labels} unique labels in the dataset.")

class_counts = full_data['label'].value_counts()
print(class_counts)

# Display the first few entries to verify the changes
full_data.describe()

['part-00000-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv', 'part-00001-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv', 'part-00002-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv', 'part-00003-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv', 'part-00004-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv', 'part-00005-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv', 'part-00006-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv', 'part-00007-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv', 'part-00008-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv', 'part-00009-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv', 'part-00010-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv', 'part-00011-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv', 'part-00012-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv', 'part-00013-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv', 'part-00014-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv', 'part-00015-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv', 'part-00016-363d1ba3-8ab5-4f96-bc25-4d5

,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,...,AVG,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight
count,1.149323e+06,1.149323e+06,1.149323e+06,1.149323e+06,1.149323e+06,1.149323e+06,1.149323e+06,1.149323e+06,1.149323e+06,1.149323e+06,...,1.149323e+06,1.149323e+06,1.149323e+06,1.149323e+06,1.149323e+06,1.149323e+06,1.149323e+06,1.149323e+06,1.149323e+06,1.149323e+06
mean,5.639886e+00,7.603552e+04,9.066000e+00,6.633707e+01,9.227045e+03,9.227045e+03,3.665947e-06,8.637259e-02,2.067147e-01,9.023573e-02,...,1.248018e+02,3.347234e+01,1.248674e+02,8.318251e+07,9.498405e+00,1.312644e+01,4.730562e+01,3.108634e+04,9.647052e-02,1.415101e+02
std,2.634897e+02,4.552386e+05,8.941014e+00,1.397396e+01,1.012008e+05,1.012008e+05,2.072376e-03,2.809136e-01,4.049493e-01,2.865193e-01,...,2.410931e+02,1.613550e+02,2.419796e+02,1.701815e+07,8.175700e-01,8.636226e+00,2.282203e+02,3.273478e+05,2.330712e-01,2.102859e+01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,4.200000e+01,0.000000e+00,4.200000e+01,0.000000e+00,1.000000e+00,9.165151e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
25%,0.000000e+00,5.400000e+01,6.000000e+00,6.400000e+01,2.099143e+00,2.099143e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,5.000000e+01,0.000000e+00,5.000000e+01,8.307156e+07,9.500000e+00,1.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,1.415500e+02
50%,0.000000e+00,5.400000e+01,6.000000e+00,6.400000e+01,1.581202e+01,1.581202e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,5.400000e+01,0.000000e+00,5.400000e+01,8.312452e+07,9.500000e+00,1.039230e+01,0.000000e+00,0.000000e+00,0.000000e+00,1.415500e+02
75%,1.053554e-01,3.050300e+02,1.464000e+01,6.400000e+01,1.185152e+02,1.185152e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,5.405011e+01,3.728500e-01,5.406000e+01,8.334390e+07,9.500000e+00,1.039673e+01,5.059213e-01,1.344812e+00,8.000000e-02,1.415500e+02
max,5.946646e+04,9.810438e+06,4.700000e+01,2.550000e+02,7.340032e+06,7.340032e+06,2.059601e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,7.959452e+03,8.026920e+03,5.858000e+03,1.676394e+08,1.450000e+01,1.236001e+02,1.135178e+04,7.370417e+07,1.000000e+00,2.446000e+02


# Preprocessing
## Duplicating data for classes

In [38]:
all_data = {}

for _class in apply_classes:
    all_data[_class] = full_data.copy()
    
    match _class:            
        case '8+1':
            label_categories = {
                'Backdoor_Malware': 'Web',
                'BenignTraffic': 'Benign',
                'BrowserHijacking': 'Web',
                'CommandInjection': 'DDoS',
                'DDoS-ACK_Fragmentation': 'DDoS',
                'DDoS-HTTP_Flood': 'DDoS',
                'DDoS-ICMP_Flood': 'DDoS',
                'DDoS-ICMP_Fragmentation': 'DDoS',
                'DDoS-PSHACK_Flood': 'DDoS',
                'DDoS-RSTFINFlood': 'DDoS',
                'DDoS-SYN_Flood': 'DDoS',
                'DDoS-SlowLoris': 'DDoS',
                'DDoS-SynonymousIP_Flood': 'DDoS',
                'DDoS-TCP_Flood': 'DDoS',
                'DDoS-UDP_Flood': 'DDoS',
                'DDoS-UDP_Fragmentation': 'DDoS',
                'DNS_Spoofing': 'Spoofing',
                'DictionaryBruteForce': 'BruteForce',
                'DoS-HTTP_Flood': 'DoS',
                'DoS-SYN_Flood': 'DoS',
                'DoS-TCP_Flood': 'DoS',
                'DoS-UDP_Flood': 'DoS',
                'MITM-ArpSpoofing': 'Spoofing',
                'Mirai-greeth_flood': 'Mirai',
                'Mirai-greip_flood': 'Mirai',
                'Mirai-udpplain': 'Mirai',
                'Recon-HostDiscovery': 'Recon',
                'Recon-OSScan': 'Recon',
                'Recon-PingSweep': 'Recon',
                'Recon-PortScan': 'Recon',
                'SqlInjection': 'Web',
                'Uploading_Attack': 'Web',
                'VulnerabilityScan': 'Recon',
                'XSS': 'Web'
            }
            all_data['8+1']['label'] = all_data['8+1']['label'].map(label_categories)
            
        case '1+1':
            all_data['1+1'].loc[all_data['1+1']['label'] != 'BenignTraffic', 'label'] = 'Attack'
            all_data['1+1'].loc[all_data['1+1']['label'] == 'BenignTraffic', 'label'] = 'Benign'

all_data[apply_classes[0]].head()

,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label
135011,4.223898,123.92,6.11,64.0,2.475843,2.475843,0.0,0.0,1.0,0.0,...,2.549096,55.16,8.297300e+07,9.5,10.454952,3.613746,72.944002,0.09,141.55,DoS-SYN_Flood
203161,1.081191,88.56,6.00,64.0,1.185178,1.185178,0.0,0.0,1.0,0.0,...,0.000000,54.00,8.336560e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,DDoS-SynonymousIP_Flood
121081,0.000000,54.00,6.00,64.0,24.204573,24.204573,0.0,0.0,0.0,0.0,...,0.000000,54.00,8.307624e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,DDoS-TCP_Flood
164680,0.000000,54.00,6.00,64.0,16.182906,16.182906,0.0,1.0,0.0,1.0,...,0.000000,54.00,8.334499e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,DDoS-RSTFINFlood
72705,152.665709,12591.40,7.60,106.3,0.845050,0.845050,0.0,0.0,0.0,0.0,...,84.022061,126.00,1.668469e+08,13.5,15.437371,119.114693,7102.458425,1.00,244.60,Recon-HostDiscovery


In [39]:
all_data[apply_classes[1]].head()

,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label
135011,4.223898,123.92,6.11,64.0,2.475843,2.475843,0.0,0.0,1.0,0.0,...,2.549096,55.16,8.297300e+07,9.5,10.454952,3.613746,72.944002,0.09,141.55,DoS
203161,1.081191,88.56,6.00,64.0,1.185178,1.185178,0.0,0.0,1.0,0.0,...,0.000000,54.00,8.336560e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,DDoS
121081,0.000000,54.00,6.00,64.0,24.204573,24.204573,0.0,0.0,0.0,0.0,...,0.000000,54.00,8.307624e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,DDoS
164680,0.000000,54.00,6.00,64.0,16.182906,16.182906,0.0,1.0,0.0,1.0,...,0.000000,54.00,8.334499e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,DDoS
72705,152.665709,12591.40,7.60,106.3,0.845050,0.845050,0.0,0.0,0.0,0.0,...,84.022061,126.00,1.668469e+08,13.5,15.437371,119.114693,7102.458425,1.00,244.60,Recon


In [40]:
all_data[apply_classes[2]].head()

,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label
135011,4.223898,123.92,6.11,64.0,2.475843,2.475843,0.0,0.0,1.0,0.0,...,2.549096,55.16,8.297300e+07,9.5,10.454952,3.613746,72.944002,0.09,141.55,Attack
203161,1.081191,88.56,6.00,64.0,1.185178,1.185178,0.0,0.0,1.0,0.0,...,0.000000,54.00,8.336560e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,Attack
121081,0.000000,54.00,6.00,64.0,24.204573,24.204573,0.0,0.0,0.0,0.0,...,0.000000,54.00,8.307624e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,Attack
164680,0.000000,54.00,6.00,64.0,16.182906,16.182906,0.0,1.0,0.0,1.0,...,0.000000,54.00,8.334499e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,Attack
72705,152.665709,12591.40,7.60,106.3,0.845050,0.845050,0.0,0.0,0.0,0.0,...,84.022061,126.00,1.668469e+08,13.5,15.437371,119.114693,7102.458425,1.00,244.60,Attack



## Encoding Labels

In [41]:
for _class in apply_classes:    
    match _class:
        case '33+1':
            full_label_encoder = LabelEncoder()
            all_data['33+1']['label'] = full_label_encoder.fit_transform(all_data['33+1']['label'])
            
        case '8+1':
            class_label_encoder = LabelEncoder()
            all_data['8+1']['label'] = class_label_encoder.fit_transform(all_data['8+1']['label'])
            
        case '1+1':
            binary_label_encoder = LabelEncoder()
            all_data['1+1']['label'] = binary_label_encoder.fit_transform(all_data['1+1']['label'])
            

# Store label mappings
label_mapping = {index: label for index, label in enumerate(full_label_encoder.classes_)}
print("Label mappings:", label_mapping)

# Retrieve the numeric codes for classes
class_codes = {label: full_label_encoder.transform([label])[0] for label in full_label_encoder.classes_}

# Print specific instances after label encoding
print("After encoding:")
for label, code in class_codes.items():
    # Print the first instance of each class
    print(f"First instance of {label} (code {code}):")
    print(all_data[apply_classes[0]][all_data[apply_classes[0]]['label'] == code].iloc[0])

all_data[apply_classes[0]].head()

Label mappings: {0: 'Backdoor_Malware', 1: 'BenignTraffic', 2: 'BrowserHijacking', 3: 'CommandInjection', 4: 'DDoS-ACK_Fragmentation', 5: 'DDoS-HTTP_Flood', 6: 'DDoS-ICMP_Flood', 7: 'DDoS-ICMP_Fragmentation', 8: 'DDoS-PSHACK_Flood', 9: 'DDoS-RSTFINFlood', 10: 'DDoS-SYN_Flood', 11: 'DDoS-SlowLoris', 12: 'DDoS-SynonymousIP_Flood', 13: 'DDoS-TCP_Flood', 14: 'DDoS-UDP_Flood', 15: 'DDoS-UDP_Fragmentation', 16: 'DNS_Spoofing', 17: 'DictionaryBruteForce', 18: 'DoS-HTTP_Flood', 19: 'DoS-SYN_Flood', 20: 'DoS-TCP_Flood', 21: 'DoS-UDP_Flood', 22: 'MITM-ArpSpoofing', 23: 'Mirai-greeth_flood', 24: 'Mirai-greip_flood', 25: 'Mirai-udpplain', 26: 'Recon-HostDiscovery', 27: 'Recon-OSScan', 28: 'Recon-PingSweep', 29: 'Recon-PortScan', 30: 'SqlInjection', 31: 'Uploading_Attack', 32: 'VulnerabilityScan', 33: 'XSS'}
After encoding:
First instance of Backdoor_Malware (code 0):
flow_duration      1.341747e+03
Header_Length      4.718860e+04
Protocol Type      1.480000e+01
Duration           1.000000e+02
Rate

,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label
135011,4.223898,123.92,6.11,64.0,2.475843,2.475843,0.0,0.0,1.0,0.0,...,2.549096,55.16,8.297300e+07,9.5,10.454952,3.613746,72.944002,0.09,141.55,19
203161,1.081191,88.56,6.00,64.0,1.185178,1.185178,0.0,0.0,1.0,0.0,...,0.000000,54.00,8.336560e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,12
121081,0.000000,54.00,6.00,64.0,24.204573,24.204573,0.0,0.0,0.0,0.0,...,0.000000,54.00,8.307624e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,13
164680,0.000000,54.00,6.00,64.0,16.182906,16.182906,0.0,1.0,0.0,1.0,...,0.000000,54.00,8.334499e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,9
72705,152.665709,12591.40,7.60,106.3,0.845050,0.845050,0.0,0.0,0.0,0.0,...,84.022061,126.00,1.668469e+08,13.5,15.437371,119.114693,7102.458425,1.00,244.60,26


## X, y Splitting

In [42]:
X = {}
y = {}

for _class in apply_classes:
    X[_class] = all_data[_class].drop('label', axis=1)
    y[_class] = all_data[_class]['label']

print(f'X: {X[apply_classes[0]].shape}, y: {y[apply_classes[0]].shape}')

X: (1149323, 46), y: (1149323,)


# Sampling

In [ ]:
if apply_sampling is not None:
    
    sampler = None
    
    match apply_sampling:
        case 'RandomOverSampler':
            from imblearn.over_sampling import RandomOverSampler
            sampler = RandomOverSampler(random_state=42)
            
        case 'RandomUnderSampler':
            from imblearn.under_sampling import RandomUnderSampler
            sampler = RandomUnderSampler(random_state=42)
            
        case 'SMOTE':
            from imblearn.over_sampling import SMOTENC
            cat_cols = [
                'Protocol Type', 'Drate', 'fin_flag_number', 'syn_flag_number', 'rst_flag_number',
                'psh_flag_number', 'ack_flag_number', 'ece_flag_number',
                'cwr_flag_number', 'HTTP', 'HTTPS', 'DNS', 'Telnet',
                'SMTP', 'SSH', 'IRC', 'TCP', 'UDP', 'DHCP', 'ARP',
                'ICMP', 'IPv', 'LLC'
            ]
            sampler = SMOTENC(categorical_features=cat_cols, random_state=42, n_jobs=-1)
        case 'ClusterCentroids':
            from imblearn.under_sampling import ClusterCentroids
            sampler = ClusterCentroids(random_state=42)
    
    for _class in apply_classes:
        print(f'{datetime.now()}: Applying undersampling to {_class} label grouping...')
        X[_class], y[_class] = sampler.fit_resample(X[_class], y[_class])
        print(f'{_class} : X: {X[_class].shape}, y: {y[_class].shape}')
    
    print(f'X: {X[apply_classes[0]].shape}, y: {y[apply_classes[0]].shape}')
else:
    print('No sampling selected.')

2024-05-09 19:38:05.249731: Applying undersampling to 33+1 label grouping...


C:\ProgramData\anaconda3\envs\sampling\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\ProgramData\anaconda3\envs\sampling\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\ProgramData\anaconda3\envs\sampling\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\ProgramData\anaconda3\envs\sampling\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: User

33+1 : X: (1156, 46), y: (1156,)
2024-05-09 19:38:10.049416: Applying undersampling to 8+1 label grouping...


C:\ProgramData\anaconda3\envs\sampling\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


8+1 : X: (1156, 46), y: (1156,)
2024-05-09 19:38:42.132717: Applying undersampling to 1+1 label grouping...


In [ ]:
# Recombine the resampled features and labels back
all_data_resampled = {}
for _class in apply_classes:
    all_data_resampled[_class] = pd.concat([X[_class], y[_class]], axis=1)

print("Resampled Data (UNSCALED):")
for label, code in class_codes.items():
    # Print the first instance of each class
    print(f"First instance of {label} (code {code}):")
    print(all_data_resampled[apply_classes[0]][all_data_resampled[apply_classes[0]]['label'] == code].iloc[0])

## Real vs Resampled Dataset Analysis

In [ ]:
all_data_resampled[apply_classes[0]].describe()

### Generate Reports

In [ ]:
for _class in apply_classes:
    original_report = ProfileReport(all_data[_class], title=f'{_class} Original Data', minimal=True)
    resampled_report = ProfileReport(all_data_resampled[_class], title=f'{_class} Resampled Data', minimal=True)
    comparison_report = original_report.compare(resampled_report)
    comparison_report.to_file(f'./profile_reports/{apply_sampling}_{_class}_resampling_report.html')

# Evaluator Model

## Imports

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

## Preprocessing
### Scaling Numerical Features

In [ ]:
num_cols = [
    'flow_duration', 'Header_Length',  'Duration', 'Rate', 'Srate', 'ack_count', 'syn_count', 'fin_count',
    'urg_count', 'rst_count', 'Tot sum', 'Min', 'Max', 'AVG', 'Std', 'Tot size', 'IAT', 'Number', 'Magnitue',
    'Radius', 'Covariance', 'Variance', 'Weight'
]

scaler = StandardScaler()
for _class in apply_classes:
    all_data_resampled[_class][num_cols] = scaler.fit_transform(all_data_resampled[_class][num_cols])

### X, y Train/Test Splitting

In [ ]:
X_train = {}
X_test = {}
y_train = {}
y_test = {}
    
for _class in apply_classes:
    X = all_data_resampled[_class].drop('label', axis=1)
    y = all_data_resampled[_class]['label']
    
    X_train_temp, X_test_temp, y_train_temp, y_test_temp = train_test_split(X, y, test_size=0.2, random_state=42)
    
    X_train.update({_class: X_train_temp})
    X_test.update({_class: X_test_temp})
    y_train.update({_class: y_train_temp})
    y_test.update({_class: y_test_temp})
    
print(f'X_train: {X_train[apply_classes[0]].shape}, y_train: {y_train[apply_classes[0]].shape}, X_test: {X_test[apply_classes[0]].shape}, y_test: {y_test[apply_classes[0]].shape}')

## Training

In [ ]:
for evaluator_type in apply_evaluators:
    match evaluator_type:
        case 'XGBoost':
            from xgboost import XGBClassifier
            evaluator = XGBClassifier()
        case 'LogisticRegression':
            from sklearn.linear_model import LogisticRegression
            evaluator = LogisticRegression(random_state=42, n_jobs=-1)
        case 'Perceptron':
            from sklearn.linear_model import Perceptron
            evaluator = Perceptron(random_state=42, n_jobs=-1)
        case 'AdaBoost':
            from sklearn.ensemble import AdaBoostClassifier
            evaluator = AdaBoostClassifier(random_state=42, algorithm='SAMME')
        case 'RandomForest':
            from sklearn.ensemble import RandomForestClassifier
            evaluator = RandomForestClassifier(random_state=42, n_jobs=-1)
        case 'DeepNeuralNetwork':
            from sklearn.neural_network import MLPClassifier
            evaluator = MLPClassifier(random_state=42)
        case 'KNearestNeighbor':
            from sklearn.neighbors import KNeighborsClassifier
            evaluator = KNeighborsClassifier(n_jobs=-1)
        case _:
            print(f'Invalid evaluator model: {evaluator_type}')
    
    
    
    for _class in apply_classes:
        # XGBoost for binary classification must be a binary objective
        if evaluator_type == 'XGBoost' and _class == '1+1':
            evaluator = XGBClassifier(objective='binary:logistic')
            
        print(f'{datetime.now()} : Training {evaluator_type} on {apply_sampling} balanced data with {_class} label classes')
        evaluator.fit(X_train[_class], y_train[_class])
    
        print(f'{datetime.now()} : Predicting {evaluator_type} on {_class} classes')
        y_pred = evaluator.predict(X_test[_class])
    
        print(f'{evaluator_type} {_class} Metrics')
        print(f'   Accuracy: {accuracy_score(y_test[_class], y_pred)}')
        print(f'   Precision: {precision_score(y_test[_class], y_pred, average='weighted', zero_division=0.0)}')
        print(f'   Recall: {recall_score(y_test[_class], y_pred, average='weighted')}')
        print(f'   F1: {f1_score(y_test[_class], y_pred, average='weighted')}')
        print()

## Model Analysis

In [ ]:
# Disabled, needs multi-label-group and multi-estimator re-implementation
# 
# from sklearn.metrics import confusion_matrix
# 
# cm = pd.DataFrame(confusion_matrix(y_test, y_pred), columns = full_label_encoder.classes_)
# cm.insert(0, column='Actual', value=full_label_encoder.classes_)
# cm